In [1]:
from time import sleep
import pandas as pd
import requests
import json

def get_api(address):
  x = requests.get('http://api-ropsten.etherscan.io/api?module=account&action=tokentx&address='+address+'&startblock=0&endblock=999999999&sort=asc&apikey=FV2TJTM9U8WYF4TA56DE9UIBYN4ZGWM198', headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'})
  return json.loads(x.content)

In [2]:
df = pd.DataFrame([], columns=['hash','from','to','value'])
user_address = '0xEcA19B1a87442b0c25801B809bf567A6ca87B1da'
user_transactions = [t for t in get_api(user_address)['result'] if t['tokenSymbol'] == 'BKTC' and t['from'] == user_address.lower()]
user_transactions = [(t['hash'], t['from'], t['to'], t['value'], t['timeStamp']) for t in user_transactions]
df = df.append(pd.DataFrame(user_transactions, columns=['hash','from','to','value','timestamp']))

In [3]:
check = 0
cur_list = ['check']
while len(cur_list) > 0:
  cur_list = []
  tmp = df.copy()
  for i in range(check, len(tmp)):
    address = df['to'][i]
    time = df['timestamp'][i]
    user_transactions = []
    for t in get_api(address)['result']: 
      if type(t) == str:
        continue
      if t['tokenSymbol'] == 'BKTC' and t['from'] == address.lower() and t['timeStamp'] > time:
        user_transactions.append(t)
    user_transactions = [(t['hash'], t['from'], t['to'], t['value'], t['timeStamp']) for t in user_transactions]
    cur_list = cur_list + user_transactions
    df = df.append(pd.DataFrame(user_transactions, columns=['hash','from','to','value','timestamp'])).reset_index(drop=True)
    check = len(tmp)
    sleep(1)

In [4]:
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.iloc[:,:-1]
df['value'] = pd.to_numeric(df['value'])
df = df.drop_duplicates(['from', 'to']).reset_index(drop=True)

In [5]:
user_list = []
user_list = user_list + df['from'].tolist()
user_list = user_list + df['to'].tolist()
user_list = list(set(user_list))

In [6]:
res = []
for user in user_list:
  address = user
  cal = 0
  for t in get_api(address)['result']:
    if t['tokenSymbol'] == 'BKTC' and t['from'] == address:
      cal -= int(t['value'])
  sleep(1)
  for t in get_api(address)['result']:
    if t['tokenSymbol'] == 'BKTC' and t['to'] == address:
      cal += int(t['value'])
  sleep(1)
  res.append((user, cal))

In [7]:
res = pd.DataFrame(res, columns=['address', 'amount'])
res['amount'] = res['amount'] // 1000000000000000000

In [8]:
df.to_csv('transaction.csv')
res.to_csv('amount.csv')

In [9]:
df

,hash,from,to,value
0,0x905cdb49db7b8804830017e3f3d4a6d80f30c15f2ef8...,0xeca19b1a87442b0c25801b809bf567a6ca87b1da,0xee889cdf628e138f15538304fd72e2af56cbe4b6,10000000000000000000
1,0xde88b7bbe2d12527a53d3ce4c0f9c980da5683fd1f5e...,0xeca19b1a87442b0c25801b809bf567a6ca87b1da,0x4b41947e269464408bbd9dce7346a7e07d522b12,5000000000000000000
2,0x6ce6f2b521c84bd983752c3304ca5ae1a6b0ba54eaa2...,0xee889cdf628e138f15538304fd72e2af56cbe4b6,0x936146f6850dc157e77fc060c442e0bdffbbe6ff,7000000000000000000
3,0x455354c462770b8dd8a72952afaa61cd57b99d113436...,0xeca19b1a87442b0c25801b809bf567a6ca87b1da,0x873ce5c27fe1a679b724e25b5a856282a93b137f,5000000000000000000
4,0x54467620c2b9a81edcaf551f35971aa7d0ff70459a57...,0xee889cdf628e138f15538304fd72e2af56cbe4b6,0x11759e580b444aa1136482929984c76a19ef2794,3000000000000000000
5,0x167b71f723d9bab42391a45b0741d271dbe8885893ae...,0x936146f6850dc157e77fc060c442e0bdffbbe6ff,0x635893f9918cbb41cb6c6bde4a87e5959f53dd56,5000000000000000000
6,0xae60662ef751a8f682c78fdcf3c9ce6de57b6ec59710...,0x873ce5c27fe1a679b724e25b5a856282a93b137f,0x711447b288e2f853fa9716aa8eba37b02ca8a50f,10000000000000000000
7,0xd7fb8762f19f4745b6abd6d744df21ffd933905180ea...,0x4b41947e269464408bbd9dce7346a7e07d522b12,0x711447b288e2f853fa9716aa8eba37b02ca8a50f,3000000000000000000
8,0x55275d384f5a4a664c8b63403bd1449e5d192bb92d00...,0x711447b288e2f853fa9716aa8eba37b02ca8a50f,0x8d16abfba10c079340ddf2b252f94353cdccfa57,5000000000000000000
9,0x98daa5d5b2106c2d29dcd7d21da2102afe5ccfac57af...,0x711447b288e2f853fa9716aa8eba37b02ca8a50f,0x8f8d0e8c921b6fbee6da796f3c823a8a1e354bea,3000000000000000000


In [10]:
res

,address,amount
0,0x04bb39f36bbadf4e31aa1df09a2f5a0b76114a62,2
1,0x711447b288e2f853fa9716aa8eba37b02ca8a50f,0
2,0x8d16abfba10c079340ddf2b252f94353cdccfa57,6
3,0x8f8d0e8c921b6fbee6da796f3c823a8a1e354bea,3
4,0x936146f6850dc157e77fc060c442e0bdffbbe6ff,2
5,0x4b41947e269464408bbd9dce7346a7e07d522b12,2
6,0xeca19b1a87442b0c25801b809bf567a6ca87b1da,0
7,0xabde0e83ba513891aa3bc0a2f796132cae026106,5
8,0x11759e580b444aa1136482929984c76a19ef2794,0
9,0x635893f9918cbb41cb6c6bde4a87e5959f53dd56,5
